# Develop the R200 SHMR estimator

I will first develop the code then I'll will test on the SDSS data.

In [6]:
from astropy import units as u


In [2]:
from astropy.cosmology import FlatLambdaCDM

cosmo = FlatLambdaCDM(H0=70, Om0=0.283)
Msol = 1.98847e33
Mpc2cm = 3.086e+24
z=0.1
rhoc = cosmo.critical_density(z)

In [9]:
rhoc.to(u.Msun/u.Mpc**3)

<Quantity 1.48731815e+11 solMass / Mpc3>